In [1]:
#imports
import os
import shutil
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import random
import Augmentor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img


2025-03-02 00:11:54.832437: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-02 00:11:54.846693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740845514.856634  197616 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740845514.859412  197616 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 00:11:54.873990: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
dataPath = "/home/b1az3/code/ml/trainingData"

In [4]:
# import images
#--- words.txt ---------------------------------------------------------------#
#
# iam database word information
#
# format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A
#
#     a01-000u-00-00  -> word id for line 00 in form a01-000u
#     ok              -> result of word segmentation
#                            ok: word was correctly
#                            er: segmentation of word can be bad
#
#     154             -> graylevel to binarize the line containing this word
#     1               -> number of components for this word NOTE: dont think this exists idk why
#     408 768 27 51   -> bounding box around this word in x,y,w,h format
#     AT              -> the grammatical tag for this word, see the
#                        file tagset.txt for an explanation
#     A               -> the transcription for this word
#
class img:
    def __init__ (this,letter,p1,p2):
        this.letter = letter
        this.inPath = p1
        this.savePath = p2
    
images = []

In [5]:
#import directory for dataset 1 - IAM
directory = open(dataPath + "/ascii/words.txt","r").read().splitlines()
len(directory)
directory = directory[18:] #first 18 lines are instructions
for i in directory:
    data = i.split(" ")
    a = data[0].split("-")
    loc = "/"+a[0]+"/"+a[0]+"-"+a[1]+"/"+data[0]
    letter = data[-1]
    if len(letter) == 1:
        images.append(img(letter,"/words"+loc,"/IAM"+loc))
print(len(images))




18079


In [5]:
#To add a pixel of border around each picture, because zooming out uses the edge pixels
def addBorder (path):
    t = cv2.imread(dataPath+path+".png")
    pic = np.array(t)
    x,y,z = pic.shape #ALSO change colour here if needed
    
    column = np.array([[[255,255,255]] for i in range (x)]) # to add 1 pixel of white around the picture, because the picture is cropped to exactly where the side contains black and zoom function does not take that well
    row = np.array([[[255,255,255] for i in range (y+2)]])
    pic = np.concatenate([column,pic,column], axis=1)
    pic = np.concatenate([row,pic,row],axis=0)
    cv2.imwrite(dataPath+path+".jpg",pic)
    return pic

for img_ in images:
    try:
        addBorder(img_.inPath)
    except:
        print(dataPath+img_.inPath+".png")

[ WARN:0@0.177] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


In [6]:
#import directory for dataset 1 - EMNIST
dir = os.listdir(dataPath + "/by_class/")
for i in dir:
    letter = chr(int("0x"+i,0))
    subDir = os.listdir(dataPath + "/by_class/"+i)
    for j in subDir:
        if j[-4:] != ".mit":
            imgDir = os.listdir(dataPath + "/by_class/"+i+"/"+j)
            for k in imgDir:
                images.append(img(letter,"/by_class/"+i+"/"+j+"/"+k[:-4],"/EMNIST/"+i+"/"+j+"/"+k[:-4]))

print(len(images))


1960591


In [7]:
def augment (path,savePath):
    # Converting the input sample image to an array
    # Reshaping the input image
    datagen = ImageDataGenerator(zoom_range = [1,2],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1)
   
    # Loading a sample image 
    img = load_img(path) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape) 
    
    # Generating and saving 5 augmented samples 
    # using the above defined parameters. 
    i = 0
    if not os.path.exists(savePath):
        os.makedirs(savePath)
    for batch in datagen.flow(x, batch_size = 1,
                            save_to_dir =savePath, 
                            save_prefix ='image', save_format ='jpeg'):
        i += 1
        if i >= 5:
            break

In [ ]:
count = 0

In [9]:
print (count)
print (len(images))

1813547
1960591


In [10]:
#execute augment function, will take long time
#change
last = ""
total = len(images)
last = 0
imagesCopy = images.copy()
images = images[count:]
for img_ in images:
    count +=1
    if int(count/total*1000) > last:
        last = int(count/total*1000)
        print(count/total*100)

    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)
    

92.50006758166288
92.60003743769099
92.70000729371908
92.80002815477579
92.90004901583248
93.00001887186058
93.10003973291727
93.20000958894538
93.30003045000207
93.40000030603018
93.50002116708687
93.60004202814356
93.70001188417166
93.80003274522835
93.90000260125646
94.00002346231315
94.10004432336983
94.20001417939795
94.30003504045465
94.40000489648274
94.50002575753945
94.60004661859612
94.70001647462423
94.80003733568093
94.90000719170904
95.00002805276571
95.1000489138224
95.20001876985052
95.30003963090721
95.40000948693532
95.50003034799201
95.60000020402012
95.7000210650768
95.80004192613349
95.9000117821616
96.00003264321829
96.1000024992464
96.20002336030309
96.30004422135978
96.40001407738788
96.50003493844459
96.60000479447268
96.70002565552937
96.80004651658606
96.90001637261417
97.00003723367087
97.10000708969898
97.20002795075567
97.30004881181236
97.40001866784046
97.50003952889715
97.60000938492526
97.70003024598195
97.80000010201006
97.90002096306675
98.00004182412

In [11]:
print(len(images))

147044


In [8]:
count = 0

In [9]:
def resize(image_path,size,out_path): #resize to size and add black padding around unknown
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_pad(image, size[0],size[1])
    tf.keras.utils.save_img(out_path, image, file_format="jpeg")
last = 0
total = len(images)
for img in images:
    count +=1
    if int(count/total*1000) > last:
        last = int(count/total*1000)
        print(count/total*100)

    try:
        files = os.listdir(dataPath+"/augmentedWords"+img.savePath)
        endpath = dataPath + "/final_aug_words"+img.savePath
        if not os.path.exists(endpath):
            os.makedirs(endpath)
        for augImg in files:
            resize(dataPath+"/augmentedWords"+img.savePath+"/"+augImg,(224,224),endpath+"/"+augImg)
            os.remove(dataPath+"/augmentedWords"+img.savePath+"/"+augImg)  
        os.rmdir(dataPath+"/augmentedWords"+img.savePath)
    except:
        continue
    
#takes 46 mins

0.10002086105669158
0.20004172211338317
0.300011578141489
0.4000324391981805
0.5000022952262864
0.600023156282978
0.7000440173396696
0.8000138733677753
0.900034734424467
1.0000045904525727
1.1000254515092642
1.200046312565956
1.3000161685940617
1.4000370296507532
1.500006885678859
1.6000277467355506
1.700048607792242
1.8000184638203482
1.9000393248770397
2.0000091809051455
2.100030041961837
2.2000509030185285
2.3000207590466344
2.400041620103326
2.500011476131432
2.6000323371881233
2.7000021932162293
2.8000230542729208
2.9000439153296127
3.000013771357718


I0000 00:00:1740845534.120528  197616 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9763 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


3.1000346324144097
3.2000044884425156
3.3000253494992067
3.4000462105558986
3.5000160665840045
3.6000369276406965
3.700006783668802
3.8000276447254935
3.9000485057821854
4.000018361810291
4.100039222866982
4.200009078895088
4.30002993995178
4.400050801008471
4.500020657036577
4.600041518093269
4.700011374121375
4.800032235178066
4.900002091206172
5.000022952262864
5.100043813319555
5.200013669347661
5.300034530404353
5.4000043864324585
5.50002524748915
5.6000461085458415
5.700015964573947
5.800036825630639
5.900006681658745
6.000027542715436
6.100048403772128
6.200018259800234
6.300039120856924
6.400008976885031
6.500029837941723
6.600050698998413
6.70002055502652
6.800041416083212
6.900011272111318
7.000032133168009
7.100001989196116
7.200022850252806
7.300043711309498
7.400013567337604
7.500034428394295
7.600004284422401
7.700025145479093
7.8000460065357835
7.90001586256389
8.000036723620582
8.100006579648687
8.200027440705378
8.300048301762072
8.400018157790177
8.500039018846868
8.6

In [4]:
import shutil
shutil.rmtree(dataPath+"/augmentedWords")